In [2]:
import os
import re
import numpy as np
from glob import glob
from scipy.signal import convolve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

In [3]:
def moving_average_filter(signal, window_size=5):
    """
    Applies a moving average filter (FIR filter with uniform weights)
    to smooth the input signal.
    """
    return np.convolve(signal, np.ones(window_size)/window_size, mode='valid')

def extract_features(signal):
    """
    Extracts simple statistical features from the signal:
    mean, standard deviation, maximum, minimum, and energy.
    """
    mean = np.mean(signal)
    std = np.std(signal)
    maximum = np.max(signal)
    minimum = np.min(signal)
    energy = np.sum(signal**2)
    return np.array([mean, std, maximum, minimum, energy])


In [4]:
def load_segment_files(folder_path):
    """
    Groups CSV files in the given folder into segments by pairing the low band (L) 
    and high band (H) parts. 
    File naming convention: <segmentID><L/H>_<pairIndex>.csv
    e.g., "11000L_1.csv" is the low band and "11000H_1.csv" is the high band.
    The function loads both parts and concatenates them to form a complete segment.
    """
    segment_dict = {}
    # Regular expression to parse filenames (L for low band, H for high band)
    pattern = re.compile(r"(\d+)([LH])_(\d+)\.csv")
    # List all CSV files in the folder
    files = glob(os.path.join(folder_path, "*.csv"))
    
    for file in files:
        basename = os.path.basename(file)
        match = pattern.match(basename)
        if match:
            seg_id = match.group(1)   # e.g., "11000"
            band = match.group(2)     # "L" (low band) or "H" (high band)
            pair_idx = match.group(3) # e.g., "1"
            key = (seg_id, pair_idx)
            if key not in segment_dict:
                segment_dict[key] = {}
            segment_dict[key][band] = file
        else:
            print(f"Filename {basename} did not match expected pattern and will be skipped.")
    
    segments = []
    for key, parts in segment_dict.items():
        if 'L' in parts and 'H' in parts:
            try:
                # Load both low band and high band parts; adjust delimiter if needed
                data_low = np.loadtxt(parts['L'], delimiter=',')
                data_high = np.loadtxt(parts['H'], delimiter=',')
                # Combine the two parts by concatenation along the time axis
                segment = np.concatenate((data_low, data_high))
                segments.append(segment)
            except Exception as e:
                print(f"Error loading segment {key}: {e}")
        else:
            print(f"Skipping segment {key} because it doesn't have both low (L) and high (H) band parts.")
    return segments

In [9]:
base_dir = './Data/DroneRF'  # e.g., "/data/DroneRF"
drone_folders = [
  'Data\DroneRF\AR drone',
    'Data\DroneRF\Bepop drone',
    'Data\DroneRF\Phantom drone'
]
background_folder = os.path.join(base_dir, 'Background RF activities')

# ----- Load the Data -----
X = []
y = []


In [10]:
# Load and process drone segments (label 1 for drone present)
for folder in drone_folders:
    print(f"Loading segments from {folder}")
    segments = load_segment_files(folder)
    for segment in segments:
        # Apply moving average filter for smoothing
        filtered_signal = moving_average_filter(segment, window_size=5)
        # Normalize signal between -1 and 1
        if np.max(filtered_signal) != np.min(filtered_signal):
            norm_signal = 2 * (filtered_signal - np.min(filtered_signal)) / (np.max(filtered_signal) - np.min(filtered_signal)) - 1
        else:
            norm_signal = filtered_signal
        features = extract_features(norm_signal)
        X.append(features)
        y.append(1)

# Load and process background segments (label 0 for no drone present)


Loading segments from Data\DroneRF\AR drone
Loading segments from Data\DroneRF\Bepop drone
Error loading segment ('10000', '15'): 
Error loading segment ('10000', '16'): 
Error loading segment ('10000', '17'): 
Error loading segment ('10000', '18'): 
Error loading segment ('10000', '19'): 
Error loading segment ('10000', '20'): 
Error loading segment ('10000', '3'): 
Error loading segment ('10000', '4'): 
Error loading segment ('10000', '5'): 
Error loading segment ('10000', '6'): 
Error loading segment ('10000', '7'): 
Error loading segment ('10000', '8'): 
Error loading segment ('10000', '9'): 
Error loading segment ('10001', '0'): 
Error loading segment ('10001', '1'): 
Error loading segment ('10001', '10'): 
Error loading segment ('10001', '11'): 
Error loading segment ('10001', '12'): 
Error loading segment ('10001', '13'): 
Error loading segment ('10001', '14'): 
Error loading segment ('10001', '15'): 
Error loading segment ('10001', '16'): 
Error loading segment ('10001', '17'):

In [19]:
r=[]
z=[]
print(f"Loading segments from {background_folder}")
background_segments = load_segment_files('Data\DroneRF\Background RF activites')
for segment in background_segments:
    filtered_signal = moving_average_filter(segment, window_size=5)
    if np.max(filtered_signal) != np.min(filtered_signal):
        norm_signal = 2 * (filtered_signal - np.min(filtered_signal)) / (np.max(filtered_signal) - np.min(filtered_signal)) - 1
    else:
        norm_signal = filtered_signal
    features = extract_features(norm_signal)
    r.append(features)
    z.append(0)

Loading segments from ./Data/DroneRF\Background RF activities


In [22]:
r1 = np.array(r)
z2 = np.array(z)

In [23]:
z2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
X = np.array(X)
y = np.array(y)

In [14]:
X

array([[ 6.21589955e-03,  2.00634776e-02,  1.00000000e+00,
        -1.00000000e+00,  8.82360906e+03],
       [-2.57873564e-02,  1.63705201e-02,  1.00000000e+00,
        -1.00000000e+00,  1.86596299e+04],
       [ 3.64317034e-02,  1.82772561e-02,  1.00000000e+00,
        -1.00000000e+00,  3.32265355e+04],
       [ 4.65926416e-02,  1.80271012e-02,  1.00000000e+00,
        -1.00000000e+00,  4.99170026e+04],
       [-1.36457697e-02,  2.49172729e-02,  1.00000000e+00,
        -1.00000000e+00,  1.61415472e+04],
       [-3.33332684e-02,  1.78745033e-02,  1.00000000e+00,
        -1.00000000e+00,  2.86120872e+04],
       [ 7.02934059e-02,  1.80938459e-02,  1.00000000e+00,
        -1.00000000e+00,  1.05370982e+05],
       [-1.39480955e-02,  1.87423963e-02,  1.00000000e+00,
        -1.00000000e+00,  1.09165336e+04],
       [ 1.56227354e-02,  1.84402909e-02,  1.00000000e+00,
        -1.00000000e+00,  1.16822814e+04],
       [ 5.12768178e-02,  2.22877750e-02,  1.00000000e+00,
        -1.00000000e+00

In [20]:
z

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [21]:
r

[array([-6.09454993e-02,  1.90069018e-02,  1.00000000e+00, -1.00000000e+00,
         8.15123078e+04]),
 array([ 2.65063051e-02,  1.83262679e-02,  1.00000000e+00, -1.00000000e+00,
         2.07687220e+04]),
 array([ 1.28887417e-02,  1.83086050e-02,  1.00000000e+00, -1.00000000e+00,
         1.00264916e+04]),
 array([ 2.81738044e-02,  1.87914105e-02,  1.00000000e+00, -1.00000000e+00,
         2.29376027e+04]),
 array([-5.94850939e-04,  9.96510492e-02,  1.00000000e+00, -1.00000000e+00,
         1.98613669e+05]),
 array([ 6.39541615e-02,  2.10877240e-02,  1.00000000e+00, -1.00000000e+00,
         9.06965193e+04]),
 array([-1.07828803e-01,  1.96106256e-02,  1.00000000e+00, -1.00000000e+00,
         2.40232501e+05]),
 array([ 1.77424122e-02,  2.04079930e-02,  1.00000000e+00, -1.00000000e+00,
         1.46255844e+04]),
 array([ 3.29640386e-03,  4.45299138e-02,  1.00000000e+00, -1.00000000e+00,
         3.98755821e+04]),
 array([-3.30488639e-02,  2.83314122e-02,  1.00000000e+00, -1.00000000e+0

In [24]:
X = np.concatenate((X, r1), axis=0)
y = np.concatenate((y, z2), axis=0)

In [25]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [26]:
print("Feature matrix shape:", X.shape)
print("Labels shape:", y.shape)

Feature matrix shape: (155, 5)
Labels shape: (155,)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----- Train a Binary Classifier -----
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# ----- Save the Model to a Pickle File -----
with open('drone_detector.pkl', 'wb') as f:
    pickle.dump(clf, f)

print("Trained model saved as 'drone_detector.pkl'")

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.56      0.62         9
           1       0.83      0.91      0.87        22

    accuracy                           0.81        31
   macro avg       0.77      0.73      0.75        31
weighted avg       0.80      0.81      0.80        31

Trained model saved as 'drone_detector.pkl'


In [1]:
import os
import re
import numpy as np
from glob import glob
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

# ----- Preprocessing Functions -----
def moving_average_filter(data, window_size=5):
    """
    Applies a moving average filter (FIR filter with uniform weights)
    to smooth the input data.
    """
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

def extract_dronerf_features(signal_low, signal_high, fs, n_per_seg, feat_name='SPEC', to_norm=True):
    """
    Compute PSD features for low and high bands using Welch's method,
    optionally convert to dB (if feat_name is 'SPEC'), and normalize.
    The two PSDs are concatenated to form a single feature vector.
    
    Parameters:
      signal_low: low band signal (1D numpy array)
      signal_high: high band signal (1D numpy array)
      fs: sampling rate in Hz
      n_per_seg: number of samples per segment (for Welch's method)
      feat_name: if 'PSD' returns linear PSD; if 'SPEC', converts to dB
      to_norm: if True, normalizes the final feature vector
      
    Returns:
      Feature vector (1D numpy array)
    """
    # Compute PSD for low band
    freqs_low, psd_low = signal.welch(signal_low, fs, nperseg=n_per_seg)
    # Compute PSD for high band
    freqs_high, psd_high = signal.welch(signal_high, fs, nperseg=n_per_seg)
    
    # Concatenate PSD features
    feat = np.concatenate((psd_low, psd_high))
    
    # Convert to dB if feat_name is 'SPEC'
    if feat_name == 'SPEC':
        feat = -10 * np.log10(feat + 1e-8)  # add epsilon to avoid log(0)
    
    # Normalize feature vector if required
    if to_norm and np.max(feat) != 0:
        feat = feat / np.max(feat)
    
    return feat

def load_segment_pair_files(folder_path):
    """
    Groups CSV files in the given folder into segments by pairing the low band (L)
    and high band (H) parts.
    File naming convention: <segmentID><L/H>_<pairIndex>.csv
    Returns a list of tuples: (data_low, data_high).
    """
    segment_dict = {}
    pattern = re.compile(r"(\d+)([LH])_(\d+)\.csv")
    files = glob(os.path.join(folder_path, "*.csv"))
    
    for file in files:
        basename = os.path.basename(file)
        match = pattern.match(basename)
        if match:
            seg_id = match.group(1)
            band = match.group(2)   # 'L' or 'H'
            pair_idx = match.group(3)
            key = (seg_id, pair_idx)
            if key not in segment_dict:
                segment_dict[key] = {}
            segment_dict[key][band] = file
        else:
            print(f"Filename {basename} did not match expected pattern; skipping.")
    
    segments = []
    for key, parts in segment_dict.items():
        if 'L' in parts and 'H' in parts:
            try:
                data_low = np.loadtxt(parts['L'], delimiter=',')
                data_high = np.loadtxt(parts['H'], delimiter=',')
                segments.append((data_low, data_high))
            except Exception as e:
                print(f"Error loading segment {key}: {e}")
        else:
            print(f"Skipping segment {key} because it does not have both low and high band parts.")
    return segments

# ----- Define Dataset Paths -----
# Replace these with the actual paths to your DroneRF dataset folders.
base_dir = './Data//DroneRF'  # e.g., "/data/DroneRF"
drone_folders = [
    os.path.join(base_dir, 'AR drone'),
    os.path.join(base_dir, 'Bebop drone'),
    os.path.join(base_dir, 'Phantom drone')
]
background_folder = os.path.join(base_dir, 'Background RF activities')

# ----- Parameters for Feature Extraction -----
fs = 40e6          # Sampling rate: 40 MHz
n_per_seg = 1024   # Number of samples per segment for PSD computation
window_size = 5    # Window size for moving average filter
feat_name = 'SPEC' # Use 'SPEC' to mimic DroneRFTorch (spectrum in dB)

# ----- Load and Process the Data -----
X = []
y = []

# Process drone segments (label = 1 for drone present)
for folder in drone_folders:
    print(f"Loading drone segments from {folder}")
    segment_pairs = load_segment_pair_files(folder)
    for (sig_low, sig_high) in segment_pairs:
        # Apply moving average filter to each band
        filtered_low = moving_average_filter(sig_low, window_size)
        filtered_high = moving_average_filter(sig_high, window_size)
        # Extract combined feature vector from both bands
        features = extract_dronerf_features(filtered_low, filtered_high, fs, n_per_seg, feat_name, to_norm=True)
        X.append(features)
        y.append(1)

# Process background segments (label = 0 for no drone present)
print(f"Loading background segments from {background_folder}")
background_segment_pairs = load_segment_pair_files(background_folder)
for (sig_low, sig_high) in background_segment_pairs:
    filtered_low = moving_average_filter(sig_low, window_size)
    filtered_high = moving_average_filter(sig_high, window_size)
    features = extract_dronerf_features(filtered_low, filtered_high, fs, n_per_seg, feat_name, to_norm=True)
    X.append(features)
    y.append(0)

X = np.array(X)
y = np.array(y)

print("Feature matrix shape:", X.shape)
print("Labels shape:", y.shape)

Loading drone segments from ./Data//DroneRF\AR drone
Loading drone segments from ./Data//DroneRF\Bebop drone
Loading drone segments from ./Data//DroneRF\Phantom drone
Loading background segments from ./Data//DroneRF\Background RF activities
Feature matrix shape: (102, 1026)
Labels shape: (102,)


In [5]:
r=[]
z=[]
print(f"Loading background segments from {background_folder}")
background_segment_pairs = load_segment_pair_files('Data\DroneRF\Background RF activites')
for (sig_low, sig_high) in background_segment_pairs:
    filtered_low = moving_average_filter(sig_low, window_size)
    filtered_high = moving_average_filter(sig_high, window_size)
    features = extract_dronerf_features(filtered_low, filtered_high, fs, n_per_seg, feat_name, to_norm=True)
    r.append(features)
    z.append(0)

Loading background segments from ./Data//DroneRF\Background RF activities


In [6]:
r1 = np.array(r)
z2 = np.array(z)

In [7]:
X = np.concatenate((X, r1), axis=0)
y = np.concatenate((y, z2), axis=0)

In [8]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
# ----- Train-Test Split -----
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----- Train a Binary Classifier -----
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the classifier
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# ----- Save the Model to a Pickle File -----
with open('drone_detector-2.pkl', 'wb') as f:
    pickle.dump(clf, f)

print("Trained model saved as 'drone_detector.pkl'")

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        22

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29

Trained model saved as 'drone_detector.pkl'


In [11]:
result = clf.predict(X_test)

In [12]:
result

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1])